# Algorithme EM

In [1]:
%matplotlib inline
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

c:\users\margot\miniconda3\lib\site-packages\matplotlib\__init__.py:1350: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [34]:
wine = pd.read_csv('Datasets/Whitewine.csv')

In [35]:
d = 11  #descripteurs
N =  wine.shape[0] #produits à tester
R = 5 #experts

y = np.random.rand(N,R)
x = np.random.rand(N,d)
w0 = np.random.rand(1,d)

## Initialisation

In [36]:
def init_mu(y):
	mu = []
	for i in range(0,N):
		mu.append(np.sum(y[i])/R)
	return mu

mu0 = init_mu(y)

def sigma(z):
    return 1/(1+np.exp(-z))

## Maximum Likelihood Estimator

In [37]:
def likelihood(w,y,alpha,beta) :
    prod = 1
    for i in range(1,N):
        p = sigma(x[i].dot(w.T))
        proda = 1
        for j in range(1,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        prodb = 1
        for j in range(1,R):
            prodb = prodb*beta[j]**(y[i][j])*(1-beta[j])**(1-y[i][j])
        prod = prod * (proda*p+prodb*(1-p))
    print(prod)

## Etape E

In [38]:
#E-step

def ai(alpha,y):
    a = []
    for i in range(0,N):
        proda = 1
        for j in range(0,R):
            proda = proda*alpha[j]**(y[i][j])*(1-alpha[j])**(1-y[i][j])
        a.append(proda)
    return a

def bi(beta,y):
    b = []
    for i in range(0,N):
        prodb = 1
        for j in range(0,R):
            prodb = prodb*beta[j]**(1-y[i][j])*(1-beta[j])**(y[i][j])
        b.append(prodb)
    return b

def pi(x,w):
    p = []
    for i in range(0,N):
        p.append(sigma(x[i].dot(w.T)))
    return p

def mui(a,b,p):
    mu = []
    for i in range(0,N):
        mu.append(a[i]*p[i]/(a[i]*p[i]+b[i]*(1-p[i])))
    return mu

def E_step(x,y,alpha,beta,w):
    CE = 0 #Conditionnal excepectation
    a = ai(alpha,y)
    b = bi(beta,y)
    p = pi(x,w)
    mu = mui(a,b,p)
    for i in range(0,N):
        CE += mu[i]*np.log(p[i])*a[i]+(1-mu[i])*np.log(1-p[i])*b[i]
    return mu


## Étape M

In [39]:
#M-step

def alpha_function(mu,y):
	alpha = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += mu[i]*y[i][j]
			tmp2 += mu[i]
		alpha.append(tmp1/tmp2)
	return alpha

def beta_function(mu,y):
	beta = []
	for j in range(0,R):
		tmp1 = 0
		tmp2 = 0
		for i in range(0,N):
			tmp1 += (1-mu[i])*(1-y[i][j])
			tmp2 += 1-mu[i]
		beta.append(tmp1/tmp2)
	return beta

def updateW(w,x, eta, mu):
    g = 0
    for i in range(0,N):
        g += (mu[i] - sigma(x[i].dot(w.T)))*x[i]
    
    H = np.zeros((d,d))
    for i in range(0,N):
        H -= sigma(x[i].dot(w.T))*(1-sigma(x[i].dot(w.T)))*((x[i].reshape(11,1))*(x[i].reshape(1,11)))
    w = w - eta*np.linalg.inv(H).dot(g)
    return w

In [18]:
updateW(w0,x,0.1,mu0)

array([[ 0.06473572,  0.5191612 , -0.20359285,  0.23310836,  0.55013711,
         0.55582105, -0.20456989,  0.47838278,  0.27099025,  0.41625588,
         0.50125005]])

## Itérations

In [41]:
mu = init_mu(y)
alpha = alpha_function(mu,y)
beta = beta_function(mu,y)
diff_w=10
w = w0
while (np.linalg.norm(diff_w) > 0.001) : # Limite de convergence à decider
    mu = E_step(x,y,alpha,beta,w)
    alpha = alpha_function(mu,y)
    beta = beta_function(mu,y)
    w_bis = updateW(w,x,0.1,mu)
    diff_w = w - w_bis
    w = w_bis
    print(np.linalg.norm(diff_w))

0.00237184963422
0.000778502855682
